In [28]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from IPython.display import display
from PIL import Image
from keras.preprocessing import image
import os
from sklearn.model_selection import StratifiedKFold
import os  
from tqdm import tqdm
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from keras.models import load_model

In [17]:
def create_model():
    # Initialising the CNN
    model = Sequential()

    # Convolution
    model.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'relu'))

    # Pooling
    model.add(MaxPooling2D(pool_size = (2, 2)))

    # Second convolutional layer
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    # Third convolution layer
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    # Flattening
    model.add(Flatten())

    # Full connection
    model.add(Dense(units = 64, activation = 'relu'))
    model.add(Dense(units = 2, activation = 'softmax'))

    # Compiling the CNN
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [18]:
folder_path ='/home/riteshjain/anaconda3/Notebook/Skin/training_data1/train/Benign'
images = []
labels = []
for img in os.listdir(folder_path):
    label = 0
    img = cv2.imread(folder_path+'/'+img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    images.append(img)
    labels.append(label)

In [19]:
folder_path ='/home/riteshjain/anaconda3/Notebook/Skin/training_data1/train/Malignant'
images1 = []
labels1 = []
for img in os.listdir(folder_path):
    label1 = 1
    img = cv2.imread(folder_path+'/'+img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    images1.append(img)
    labels1.append(label1)

In [20]:
images2 = np.concatenate([images, images1])
labels2 = np.concatenate([labels, labels1])

In [21]:
images3 = images2.astype(np.int)
labels3 = labels2.astype(np.int)

In [29]:
df_y = pd.factorize(labels3)[0] 

In [37]:
n_values = np.max(df_y) + 1

In [39]:
lab = np.eye(n_values)[df_y]

In [41]:
train_x, val_x, train_y, val_y = train_test_split(images3, lab, test_size=0.1)

In [42]:
model = create_model()

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
training_set = train_datagen.flow(x = train_x, y = train_y, batch_size = 32)

validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_set = validation_datagen.flow(x = val_x, y = val_y, batch_size = 32)

In [43]:
model.fit_generator(training_set, steps_per_epoch = len(training_set), epochs = 10, validation_data= validation_set,
                validation_steps = len(validation_set))

Epoch 1/10
43/43 [==============================] - 50s 1s/step - loss: 0.7235 - acc: 0.5574 - val_loss: 0.6146 - val_acc: 0.7105
Epoch 2/10
43/43 [==============================] - 47s 1s/step - loss: 0.5734 - acc: 0.6902 - val_loss: 0.5310 - val_acc: 0.6776
Epoch 3/10
43/43 [==============================] - 53s 1s/step - loss: 0.5007 - acc: 0.7439 - val_loss: 0.4897 - val_acc: 0.7697
Epoch 4/10
43/43 [==============================] - 51s 1s/step - loss: 0.4711 - acc: 0.7621 - val_loss: 0.4441 - val_acc: 0.7434
Epoch 5/10
43/43 [==============================] - 49s 1s/step - loss: 0.4572 - acc: 0.7631 - val_loss: 0.4521 - val_acc: 0.7434
Epoch 6/10
43/43 [==============================] - 51s 1s/step - loss: 0.4268 - acc: 0.7929 - val_loss: 0.4429 - val_acc: 0.7500
Epoch 7/10
43/43 [==============================] - 49s 1s/step - loss: 0.4071 - acc: 0.8026 - val_loss: 0.4735 - val_acc: 0.7500
Epoch 8/10
43/43 [==============================] - 49s 1s/step - loss: 0.4173 - acc: 0.79

In [52]:
im = cv2.imread('/home/riteshjain/Downloads/ISIC_0034301.jpg', cv2.COLOR_BGR2RGB)
im = cv2.resize(im, (224,224))

In [53]:
im = np.expand_dims(im, axis=0)

In [55]:
model.predict(im)

array([[0., 1.]], dtype=float32)

In [11]:
model.save('/home/riteshjain/anaconda3/Notebook/skin_model/my_model.h5')